#### Import the libraries and modules

In [1]:
import torch as torch

In [2]:
import syft as sy

#### Initializing the hook

In [3]:
hook = sy.TorchHook(torch)

This is done to override PyTorch’s methods to execute commands on one worker that are called on tensors controlled by the local worker. It also allows us to move tensors between workers.

#### Creating a virtual worker

Virtual workers are entities present on our local machine. They are used to model the behavior of actual workers.

In [4]:
jake = sy.VirtualWorker(hook, id="jake")
print("Jake has: " + str(jake._objects))

Jake has: {}


Here, Jake is our virtual worker which can be considered as a separate entity on a device.

#### Sending data to a VirtualWorker

Let’s send Jake some data.

In [5]:
x = torch.tensor([1, 2, 3, 4, 5])
x = x.send(jake)
print("x: " + str(x))
print("Jake has: " + str(jake._objects))

x: (Wrapper)>[PointerTensor | me:27300779835 -> jake:14742621958]
Jake has: {14742621958: tensor([1, 2, 3, 4, 5])}


When we send a tensor to Jake, we are returned a pointer to that tensor. All the operations will be executed with this pointer. This pointer holds information about the data present on another machine. Now, x is a PointTensor.

#### Retrieving data from a VirtualWorker

Use the `get()` method to get back the value of x from Jake’s device. However, by doing so, the tensor on Jake’s device gets erased.

In [6]:
x = x.get()
print("x: " + str(x))
print("Jake has: " + str(jake._objects))

x: tensor([1, 2, 3, 4, 5])
Jake has: {}


#### Calling `send()` on a PointTensor

When we send the PointTensor x (pointing to a tensor on Jake’s machine) to another worker - John, the whole chain is sent to John and a PointTensor pointing to the node on John’s device is returned. The tensor is still present on Jake’s device.

In [7]:
john = sy.VirtualWorker(hook, id="john")
x = x.send(jake)
x = x.send(john)
print("x: " + str(x))
print("John has: " + str(john._objects))
print("Jake has: " + str(jake._objects))

x: (Wrapper)>[PointerTensor | me:39094863827 -> john:87420456620]
John has: {87420456620: (Wrapper)>[PointerTensor | john:87420456620 -> jake:14742621958]}
Jake has: {14742621958: tensor([1, 2, 3, 4, 5])}


![Figure: Using the send() method on a PointTensor [Step 2].](images/1.png "Figure: Using the send() method on a PointTensor [Step 2].")

#### Clearing or removing objects from a VirtualWorker

The `clear_objects()` method removes all the objects from a worker.

In [8]:
jake.clear_objects()
john.clear_objects()
print("Jake has: " + str(jake._objects))
print("John has: " + str(john._objects))

Jake has: {}
John has: {}


#### Moving data among VirtualWorkers

Suppose we wanted to move a tensor from Jake’s machine to John’s machine. We could do this by using the `send()` method to send the ‘pointer to tensor’ to John and let him call the `get()` method. PySfyt provides a `remote_get()` method to do this. There’s also a convenience method - `move()`, to perform the operation.

In [9]:
y = torch.tensor([6, 7, 8, 9, 10]).send(jake)
y = y.move(john)
print(y)
print("Jake has: " + str(jake._objects))
print("John has: " + str(john._objects))

(Wrapper)>[PointerTensor | me:83927397290 -> john:83927397290]
Jake has: {}
John has: {83927397290: tensor([ 6,  7,  8,  9, 10])}


![Figure: Using the move() method on a PointTensor. [Step 2]](images/2.png "Figure: Using the move() method on a PointTensor. [Step 2]")

## Additive Secret Sharing

In secret sharing, we split a secret x into a multiple number of shares and distribute them among a group of secret-holders. The secret x can be constructed only when all the shares it was split into are available.   

For example, say we split x into 3 shares: x1, x2, and x3. We randomly initialize the first two shares and calculate the third share as x3 = x - (x1 + x2). We then distribute these shares among 3 secret-holders. The secret remains hidden as each individual holds onto only one share and has no idea of the total value.   

We can make it more secure by choosing the range for the value of the shares. Let Q, a large prime number, be the upper limit. Now the third share, x3, equals Q - (x1 + x2) % Q + x.   

Figure: Encrypting x in three shares.
![Figure: Encrypting x in three shares.](images/3.jpg "Figure: Encrypting x in three shares.")

In [10]:
import random

# setting Q to a very large prime number
Q = 23740629843760239486723


def encrypt(x, n_share=3):
    r"""Returns a tuple containg n_share number of shares
    obtained after encrypting the value x."""

    shares = list()
    for i in range(n_share - 1):
        shares.append(random.randint(0, Q))
    shares.append(Q - (sum(shares) % Q) + x)
    return tuple(shares)


print("Shares: " + str(encrypt(3)))

Shares: (6791356994129235077585, 3864164084470410748324, 13085108765160593660817)


The decryption process will be shares summed together modulus Q.

Figure: Decrypting x from the three shares.
![Figure: Decrypting x from the three shares.](images/4.jpg "Figure: Decrypting x from the three shares.")

In [11]:
def decrypt(shares):
    r"""Returns a value obtained by decrypting the shares."""

    return sum(shares) % Q


print("Value after decrypting: " + str(decrypt(encrypt(3))))

Value after decrypting: 3


### Homomorphic Encryption

Homomorphic encryption is a form of encryption that allows us to perform computation on encrypted operands, resulting in encrypted output. This encrypted output when decrypted matches with the result obtained by performing the same computation on the actual operands.   

The additive secret sharing technique already has a homomorphic property. If we split x into x1, x2, and x3, and y into y1, y2, and y3, then, x+y will be equal to the value obtained after decrypting the summation of the three shares: (x1+y1), (x2+y2) and (x3+y3).   


In [12]:
def add(a, b):
    r"""Returns a value obtained by adding the shares a and b."""

    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)


x, y = 6, 8
a = encrypt(x)
b = encrypt(y)
c = add(a, b)
print("Shares encrypting x: " + str(a))
print("Shares encrypting y: " + str(b))
print("Sum of shares: " + str(c))
print("Sum of original values (x + y): " + str(decrypt(c)))

Shares encrypting x: (18070203895959794850018, 6970382073964779800108, 22440673717595904323326)
Shares encrypting y: (8526073569120074630761, 161438502552287902235, 15053117772087876953735)
Sum of shares: (2855647621319629994056, 7131820576517067702343, 13753161645923541790338)
Sum of original values (x + y): 14


We are able to calculate the value of the aggregate function - addition, without knowing the values of x and y.

### Secret Sharing using PySyft

In [13]:
jake.clear_objects()
john.clear_objects()
secure_worker = sy.VirtualWorker(hook, id="secure_worker")

jake.add_workers([john, secure_worker])
john.add_workers([jake, secure_worker])
secure_worker.add_workers([jake, john])

print("Jake has: " + str(jake._objects))
print("John has: " + str(john._objects))
print("Secure_worker has: " + str(secure_worker._objects))

Jake has: {}
John has: {}
Secure_worker has: {}


The `share()` method is used to distribute the shares among several workers. Each worker specified then receives a share and has no idea of the actual value.

In [14]:
x = torch.tensor([6])
x = x.share(jake, john, secure_worker)
print("x: " + str(x))

x: (Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:93063934540 -> jake:44390902287]
	-> (Wrapper)>[PointerTensor | me:61852485002 -> john:87331675673]
	-> (Wrapper)>[PointerTensor | me:60812633796 -> secure_worker:89050776274]
	*crypto provider: me*


Figure: Encryption of x into three shares.
![Figure: Encryption of x into three shares.](images/5.png "Figure: Encryption of x into three shares.")

In [15]:
print("Jake has: " + str(jake._objects))
print("John has: " + str(john._objects))
print("Secure_worker has: " + str(secure_worker._objects))

Jake has: {44390902287: tensor([4325627502095221125])}
John has: {87331675673: tensor([151094298338526964])}
Secure_worker has: {89050776274: tensor([-4476721800433748083])}


Figure: Distributing the shares of x among 3 VirtualWorkers.
![Figure: Distributing the shares of x among 3 VirtualWorkers.](images/6.png "Figure: Distributing the shares of x among 3 VirtualWorkers.")

As you can see, x now points to the three shares present on Jake’s, John’s and Secure_worker’s machine respectively.

In [16]:
y = torch.tensor([8])
y = y.share(jake, john, secure_worker)
print(y)

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:17361666436 -> jake:95012934188]
	-> (Wrapper)>[PointerTensor | me:65398687814 -> john:31985016021]
	-> (Wrapper)>[PointerTensor | me:49487553274 -> secure_worker:43315230055]
	*crypto provider: me*


Figure: Encryption of y into 3 shares.
![Figure: Encryption of y into 3 shares.](images/7.png "Figure: Encryption of y into 3 shares.")

Figure: Distributing the shares of y among 3 VirtualWorkers.
![Figure: Distributing the shares of y among 3 VirtualWorkers.](images/8.png "Figure: Distributing the shares of y among 3 VirtualWorkers.")

In [17]:
z = x + y
print(z)

(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:58133160191 -> jake:48479574508]
	-> (Wrapper)>[PointerTensor | me:22458431653 -> john:68882182347]
	-> (Wrapper)>[PointerTensor | me:31361678034 -> secure_worker:78289547406]
	*crypto provider: me*


Notice that the value of z obtained after adding x and y is stored in the three workers’ machines. z is also encrypted.

Figure: Performing computation on encrypted inputs.
![Figure: Performing computation on encrypted inputs.](images/9.png "Figure: Performing computation on encrypted inputs.")

In [18]:
z = z.get()
print(z)

tensor([14])


The value obtained after performing addition on encrypted shares is equal to that obtained by adding the actual numbers.

Figure: Decryption of result obtained after computation on encrypted inputs.
![Figure: Decryption of result obtained after computation on encrypted inputs.](images/10.png "Figure: Decryption of result obtained after computation on encrypted inputs.")